In [1]:
import torch
import torch.nn as nn
import math

In [2]:
class InputEmbeddings(nn.Module):
    
    """
    The purpose of this class is to create an input embedding given a token id.
    
        d_model -> the dimension of the model
        vocab_size -> size of the vocabulary
        embedding -> this uses nn.Embedding API by PyTorch, and generates an embedding.
    """
    
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)
        
    
    """
        We return an embedding; additionally, we also multiply it by the square root of the dimension of the model.
    """
    
    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

In [3]:
class PositionalEncoding(nn.Module):

    """
        
        This layer takes in a positional encoding which it adds into the input embedding to encode positional information.
    
        d_model -> same as above, the dimensionality of the model.
        seq_len -> the max length of the sequence.
        dropout -> as name imples, dropout rate.
        
    """
    
    def __init__(self, d_model: int, seq_len: int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        
        # (Seq_len, d_model)
        """
        The reason we need it to be of this "dimension" is we need to generate positional encodings of said dimension.
        The reason we need it to be of seq_len is so that we have as many positional vectors
        
        """
        pe = torch.zeros(seq_len, d_model) 
        
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        
        # pe set sine for even, cosine for odd.
        pe[:, 0::2] = torch.sin(position*div_term)
        pe[:, 1::2] = torch.cos(position*div_term)
        
        # (1, Seq_len, d_model)
        # add a dimension to pe
        pe = pe.unsqueeze(0)
        
        """
            This saves pe as a buffer.
        """
        
        self.register_buffer("pe", pe)
        
    def forward(self, x):        
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)        
        return self.dropout(x)

In [4]:
class LayerNormalization(nn.Module):
    
    def __init__(self, eps: float = 10**-6):
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(1)) # multiplied
        self.bias = nn.Parameter(torch.zeros(1)) # added
        
    def forward(self, x):
        mean = x.mean(dim = -1, keepdim=True)
        std = x.std(dim = -1, keepdim=True)
        
#         Let's see what happens when we take the root of the denominator later. 
        
        return self.alpha * (x-mean) / (std + self.eps) + self.bias
    

In [5]:
class FeedForwardBlock(nn.Module):
    def __init__(self, d_model, d_ff, dropout):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, x):
        
        #(batch_size, seq_len, d_model )
        
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [6]:
class MultiHeadAttentionBlock(nn.Module):
    
    def __init__(self, d_model, num_heads, dropout):
        
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        
        assert d_model % num_heads == 0, "d_model should be divisible by number of heads!"
        
        self.dk = self.d_model // self.num_heads
        
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        
        self.w_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        
    
    @staticmethod
    def attention(query, key, value, mask, dropout):
        
        dk = query.shape[-1]
        
        attention_scores = query @ key.transpose(-2, -1) / math.sqrt(dk)
        
        if mask is not None:
            attention_scores.mask_fill(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim = -1) # This requires a review.
        
        if dropout is not None:
            attention_scores = dropout(attention_scores)
            
        return attention_scores, (attention_scores @ value)
        
        
    def forward(self, q, k, v, mask):
        
        query = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)
        
        #The dimensionality of q, k, v is: (batch_size, seq_len, d_model)
        query = query.view(query.shape[0], query.shape[1], self.num_heads, self.dk).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.num_heads, self.dk).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.num_heads, self.dk).transpose(1, 2)
        #The switched dimensions: (batch_size, num_heads, seq_len, dk)
        
        x, attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        
        self.attention_scores = attention_scores
        return self.w_o(x)

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, dropout):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization()
        
    def forward(self, x, sublayer):
        
        return x + self.dropout(sublayer(self.norm(x)))

In [8]:
class EncoderBlock(nn.Module):
    def __init__(self, self_attention_block, feed_forward_block, dropout):
        super().__init__()
        
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_blocks = nn.ModuleList([ResidualBlock(dropout) for _ in range(2)])
        
    def forward(self, x, src_mask):
        
        x = self.residual_blocks[0](x, lambda x: self_attention_block(x, x, x, src_mask))
        x = self.residual_blocks[1](x, self.feed_forward_block)
        return x

In [9]:
class Encoder(nn.Module):
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()
        
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        
        
        return self.norm(x)

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, )